In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

In [2]:

def bermudan_option_pricer(S0, K, r, sigma, T, exercise_dates, n_paths, n_steps, option_type='put'):
    """
    Price a Bermudan option using Least Squares Monte Carlo (LSM) method
    
    Parameters:
    S0 (float): Initial stock price
    K (float): Strike price
    r (float): Risk-free rate
    sigma (float): Volatility
    T (float): Time to maturity in years
    exercise_dates (list): List of possible exercise dates (as fractions of T)
    n_paths (int): Number of Monte Carlo paths
    n_steps (int): Number of time steps
    option_type (str): 'put' or 'call'
    
    Returns:
    float: Option price
    """
    dt = T / n_steps
    
    # Generate stock price paths
    Z = np.random.standard_normal((n_paths, n_steps))
    S = np.zeros((n_paths, n_steps + 1))
    S[:, 0] = S0
    
    # Simulate stock price paths
    for t in range(1, n_steps + 1):
        S[:, t] = S[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + 
                                    sigma * np.sqrt(dt) * Z[:, t-1])
    
    # Initialize exercise value matrix
    if option_type == 'put':
        exercise_value = np.maximum(K - S, 0)
    else:
        exercise_value = np.maximum(S - K, 0)
    
    # Initialize value matrix with terminal payoff
    V = exercise_value[:, -1]
    
    # Backward induction through exercise dates
    for t in range(n_steps-1, -1, -1):
        if t/n_steps in exercise_dates:
            # Current stock prices and exercise values
            X = S[:, t]
            Y = V * np.exp(-r * dt)
            
            # Fit polynomial regression
            in_the_money = exercise_value[:, t] > 0
            if sum(in_the_money) > 0:
                coeffs = np.polyfit(X[in_the_money], Y[in_the_money], 2)
                continuation_value = np.polyval(coeffs, X)
                
                # Exercise decision
                exercise = exercise_value[:, t]
                V = np.where(exercise > continuation_value, 
                            exercise,
                            V * np.exp(-r * dt))
            else:
                V = V * np.exp(-r * dt)
        else:
            V = V * np.exp(-r * dt)
    
    return np.mean(V)
  

### Test if Bermudan price approaches European price with only final exercise date

In [3]:
S0, K = 100, 100
r, sigma = 0.05, 0.2
T = 1.0
n_paths = 50000
n_steps = 252

results = {}

# Bermudan with only final exercise
berm_price = bermudan_option_pricer(S0, K, r, sigma, T, [1.0], 
                                    n_paths, n_steps, 'put')

# Black-Scholes European price
d1 = (np.log(S0/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
d2 = d1 - sigma*np.sqrt(T)
euro_price = K*np.exp(-r*T)*norm.cdf(-d2) - S0*norm.cdf(-d1)

diff = abs(berm_price - euro_price)
results['european_limit'] = {
    'test': diff < 0.5,  # Allow for Monte Carlo error
    'berm_price': berm_price,
    'euro_price': euro_price,
    'difference': diff
}
print(results['european_limit'])


{'test': np.True_, 'berm_price': np.float64(5.593411645865038), 'euro_price': np.float64(5.573526022256971), 'difference': np.float64(0.019885623608066894)}


### Test if Option Price Increase with more excercise opportunity

In [5]:
S0, K = 100, 100
r, sigma = 0.05, 0.2
T = 1.0
n_paths = 10000
n_steps = 252

# Price with different numbers of exercise dates
price_1 = bermudan_option_pricer(S0, K, r, sigma, T, [1.0], 
                                n_paths, n_steps, 'put')
price_2 = bermudan_option_pricer(S0, K, r, sigma, T, [0.5, 1.0], 
                                n_paths, n_steps, 'put')
price_4 = bermudan_option_pricer(S0, K, r, sigma, T, [0.25, 0.5, 0.75, 1.0], 
                                n_paths, n_steps, 'put')

results['monotonicity'] = {
    'test': price_1 <= price_2 <= price_4,
    'price_1_exercise': price_1,
    'price_2_exercises': price_2,
    'price_4_exercises': price_4
}
print(results['monotonicity'])

{'test': np.False_, 'price_1_exercise': np.float64(5.5737681893063895), 'price_2_exercises': np.float64(5.849665195296545), 'price_4_exercises': np.float64(5.683420723325426)}


### Test if put call Parity hold for bermudan option

In [6]:
S0, K = 100, 100
r, sigma = 0.05, 0.2
T = 1.0
exercise_dates = [0.25, 0.5, 0.75, 1.0]
n_paths = 10000
n_steps = 252

put_price = bermudan_option_pricer(S0, K, r, sigma, T, exercise_dates, 
                                    n_paths, n_steps, 'put')
call_price = bermudan_option_pricer(S0, K, r, sigma, T, exercise_dates, 
                                    n_paths, n_steps, 'call')

# For American/Bermudan options, put-call parity is an inequality
parity_diff = call_price - put_price - S0 + K*np.exp(-r*T)
results['put_call_parity'] = {
    'test': abs(parity_diff) < 1.0,  # Allow for Monte Carlo error
    'put_price': put_price,
    'call_price': call_price,
    'parity_difference': parity_diff
}
print(results['put_call_parity'])

{'test': np.True_, 'put_price': np.float64(5.384024728516284), 'call_price': np.float64(10.012858104816297), 'parity_difference': np.float64(-0.24822417362858573)}
